In [112]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional


USER_ID = 353
PATH_DATA = '../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT

data = pd.read_csv(PATH_USER_DATA, header=None)
data = data.to_numpy()[:,1:]  # Discard weekday, just for now
print(data[0:3,:])

[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]]


In [113]:
print(data.shape)
X = data[0:-1,:]
y = data[1:,:]

X_train = X[0:190,:]
y_train = y[0:190,:]

X_test = X[190:,:]
y_test = y[190:,:]

sequence_length = 3
train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=16)
test_generator = TimeseriesGenerator(X_test, y_test, length=3, batch_size=4)

(240, 2)


In [114]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(sequence_length, 2), return_sequences=True))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(2))
lstm_model.compile(optimizer='adam', loss='mse')

epochs = 70
lstm_history = lstm_model.fit(train_generator, epochs=epochs, verbose=1, batch_size=4)

Epoch 1/70
12/12 [==============================] - 3s 5ms/step - loss: 771.3391
Epoch 2/70
12/12 [==============================] - 0s 4ms/step - loss: 709.6717
Epoch 3/70
12/12 [==============================] - 0s 4ms/step - loss: 618.7500
Epoch 4/70
12/12 [==============================] - 0s 4ms/step - loss: 525.5399
Epoch 5/70
12/12 [==============================] - 0s 4ms/step - loss: 458.6472
Epoch 6/70
12/12 [==============================] - 0s 4ms/step - loss: 408.9746
Epoch 7/70
12/12 [==============================] - 0s 4ms/step - loss: 372.7759
Epoch 8/70
12/12 [==============================] - 0s 4ms/step - loss: 344.7881
Epoch 9/70
12/12 [==============================] - 0s 4ms/step - loss: 322.0008
Epoch 10/70
12/12 [==============================] - 0s 4ms/step - loss: 301.5216
Epoch 11/70
12/12 [==============================] - 0s 4ms/step - loss: 283.4585
Epoch 12/70
12/12 [==============================] - 0s 4ms/step - loss: 266.4388
Epoch 13/70
12/12 [======

In [115]:
lstm_score = lstm_model.evaluate(test_generator, verbose=0)

print(lstm_score)

3.475886106491089
